In [1]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

## Settings
%matplotlib inline
#pd.describe_option('display')
pd.set_option('display.max_columns', None)  
pd.set_option('display.min_rows', 1) # combined with max_rows to show all rows 
pd.set_option('display.max_rows', None) 
pd.options.display.float_format = '{:.2f}'.format


## Read in data

In [2]:
# importing the training data
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

In [3]:
print(df_train.shape)
print(df_test.shape)

df_train.drop(['Id'], axis=1, inplace=True)
df_test.drop(['Id'], axis=1, inplace=True)

(1460, 81)
(1459, 80)


### Combine Train/Test for preprocessing

In [4]:
df = pd.concat((df_train, df_test), axis= 0, join = 'outer', ignore_index=True)

In [5]:
print(df.shape)
#df[['SalePrice', 'LogPrice']].tail()

(2919, 80)


In [6]:
print(df.dtypes.astype(str).value_counts())

object     43
int64      25
float64    12
dtype: int64


## Feature Engineering

### Ordinal cols-numeric

In [7]:
# features to be converted
to_cat = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual',
          'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC', 'GarageFinish', 'LandSlope', 
         'BsmtFinType1', 'BsmtFinType2', 'BsmtExposure']
        
# keys for conversions
cat_key = {'five_NA':
           {'Gd':4, 'TA':3, 'Ex':5, 'Po': 1, 'Fa':2, 'N':0, np.NaN:0},
           'finish':
           {'RFn':2, 'Unf':1, 'Fin':3, np.NaN:0},
           'slope':
           {'Gtl':3, 'Mod':2, 'Sev':1},
          'bsmt':
          {'GLQ':6, 'ALQ':5, 'BLQ':4, 'Rec':3, 'LwQ':2, 'Unf':1, np.NaN:0},
           'bsmtexp':
           {'Gd':3, 'Av':2, 'Mn': 1, 'No':0, np.NaN:0}}

# assigning each column a key dict
num_key = {'ExterQual':'five_NA', 'ExterCond':'five_NA', 'BsmtQual':'five_NA',
           'BsmtCond':'five_NA', 'HeatingQC':'five_NA', 'KitchenQual':'five_NA',
           'FireplaceQu':'five_NA', 'GarageQual':'five_NA', 'BsmtExposure':'bsmtexp',
           'GarageCond':'five_NA', 'PoolQC':'five_NA', 'GarageFinish':'finish', 
           'LandSlope': 'slope', 'BsmtFinType1':'bsmt', 'BsmtFinType2':'bsmt'}

# converting
for c in to_cat:
    p = str(f'{c}_num')
    df[p] = df[c].map(lambda x: cat_key[num_key[c]][x])

In [8]:
#df.GarageYrBlt.astype(int)
# features to be converted
to_age = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']

# new feature names
age_names = ['House_Age', 'RemodAdd_Age', 'Garage_Age', 'YrSold_Age']

# dictionary of old names and new names
ager = dict(zip(to_age,age_names))

# convert by subtracting from current year
for n, v in ager.items():
    df[v] = df[n].map(lambda x: 2020-x)

In [9]:
df.drop([c for c in to_age + to_cat], axis=1, inplace=True)

In [10]:
print(df.dtypes.astype(str).value_counts())

int64      40
object     28
float64    12
dtype: int64


### Imputations

In [11]:
nulls = {'col':[], 'null':[]}
for c in df.columns:
    if df[c].isnull().sum() > 0:
        nulls['col'].append(c)
        nulls['null'].append(df[c].isnull().sum())
        
pd.DataFrame(nulls).sort_values('null', ascending=False)

,col,null
20,MiscFeature,2814
2,Alley,2721
19,Fence,2348
22,SalePrice,1459
1,LotFrontage,486
23,Garage_Age,159
16,GarageType,157
6,MasVnrType,24
7,MasVnrArea,23
0,MSZoning,4


In [12]:
df['LotFrontage'] = df.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))

In [13]:
for name in ['Alley', 'Fence', 'GarageType', 'MasVnrType']:
    df[name] = df[name].fillna('None')
    
Zero_col = ['BsmtFinSF1','BsmtFinSF2','BsmtFullBath','BsmtHalfBath','BsmtUnfSF','TotalBsmtSF','GarageCars','GarageArea']
for name in Zero_col:
    df[name] = df[name].fillna(0)

In [14]:
df[['Garage_Age','MasVnrArea','MSZoning']] = df.groupby(['Neighborhood'])[['Garage_Age','MasVnrArea','MSZoning']].transform(lambda x: x.fillna(x.mode()[0]))

In [15]:
df.drop(['MiscFeature'], axis = 1, inplace = True)
#df.drop(['Utilities'], axis = 1, inplace = True)


In [16]:
df['LogPrice'] = np.log(df['SalePrice'])

In [17]:
nulls = {'col':[], 'null':[]}
for c in df.columns:
    if df[c].isnull().sum() > 0:
        nulls['col'].append(c)
        nulls['null'].append(df[c].isnull().sum())
        
pd.DataFrame(nulls).sort_values('null', ascending=False)

,col,null
6,SalePrice,1459
7,LogPrice,1459
0,Utilities,2
4,Functional,2
1,Exterior1st,1
2,Exterior2nd,1
3,Electrical,1
5,SaleType,1


In [18]:
df.head()
df.shape

(2919, 80)

In [19]:
print(df.dtypes.astype(str).value_counts())

int64      40
object     27
float64    13
dtype: int64


### Cat cols-Dummify

In [20]:
# ordinal dummies
dummy_list = ['MSSubClass','MSZoning','Street','Alley','LotShape','LandContour','Utilities','LotConfig','Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','Foundation','Heating','CentralAir','Electrical','GarageType','PavedDrive','Fence','SaleType','SaleCondition']
len(dummy_list)

27

In [21]:
t = df.columns.to_list()
t2 = list(set(t) - set(dummy_list))
print(len(dummy_list))
len(t2)

27


53

In [22]:
df[dummy_list].dtypes

MSSubClass        int64
MSZoning         object
Street           object
Alley            object
LotShape         object
LandContour      object
Utilities        object
LotConfig        object
Neighborhood     object
Condition1       object
Condition2       object
BldgType         object
HouseStyle       object
RoofStyle        object
RoofMatl         object
Exterior1st      object
Exterior2nd      object
MasVnrType       object
Foundation       object
Heating          object
CentralAir       object
Electrical       object
GarageType       object
PavedDrive       object
Fence            object
SaleType         object
SaleCondition    object
dtype: object

In [23]:
# Check nulls
nulls = {'col':[], 'null':[]}
for c in df[t].columns:
    if df[c].isnull().sum() > 0:
        nulls['col'].append(c)
        nulls['null'].append(df[c].isnull().sum())
        
pd.DataFrame(nulls).sort_values('null', ascending=False)


#df[t].dtypes

,col,null
6,SalePrice,1459
7,LogPrice,1459
0,Utilities,2
4,Functional,2
1,Exterior1st,1
2,Exterior2nd,1
3,Electrical,1
5,SaleType,1


In [24]:
# MSSubClass to string
df.MSSubClass = df.MSSubClass.astype('str')

In [25]:
# Number of classes before dummify (may need to combine)
for c in df[dummy_list].columns:
    print(c, df[c].nunique())

MSSubClass 16
MSZoning 5
Street 2
Alley 3
LotShape 4
LandContour 4
Utilities 2
LotConfig 5
Neighborhood 25
Condition1 9
Condition2 8
BldgType 5
HouseStyle 8
RoofStyle 6
RoofMatl 8
Exterior1st 15
Exterior2nd 16
MasVnrType 4
Foundation 6
Heating 6
CentralAir 2
Electrical 5
GarageType 7
PavedDrive 3
Fence 5
SaleType 9
SaleCondition 6


In [26]:
df.loc[df['Exterior1st'].isin(['Stone','BrkComm','CBlock','AsphShn','ImStucc']),'Exterior1st'] = 'Other'
df.loc[df['Exterior2nd'].isin(['Stone','Brk Cmn','CBlock','AsphShn','ImStucc']),'Exterior2nd'] = 'Other'
df.loc[df['Condition1'].isin(['RRAe','PosA','RRNn','RRNe']),'Condition1'] = 'Other'
df.loc[df['Condition2'].isin(['Artery','RRAe','PosA','RRNn','RRNe','RRAn']),'Condition2'] = 'Other'
df.loc[df['Neighborhood'].isin(['MeadowV','Blmngtn','BrDale','Veenker','NPkVill','Blueste']),'Neighborhood'] = 'Other'

In [27]:
# Make dummies
df1 = pd.get_dummies(df,drop_first=True)
print(df1.shape)
df1.head()

(2919, 204)


,LotFrontage,LotArea,OverallQual,OverallCond,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,SalePrice,ExterQual_num,ExterCond_num,BsmtQual_num,BsmtCond_num,HeatingQC_num,KitchenQual_num,FireplaceQu_num,GarageQual_num,GarageCond_num,PoolQC_num,GarageFinish_num,LandSlope_num,BsmtFinType1_num,BsmtFinType2_num,BsmtExposure_num,House_Age,RemodAdd_Age,Garage_Age,YrSold_Age,LogPrice,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,Alley_None,Alley_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_NoSeWa,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_Other,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Condition1_Feedr,Condition1_Norm,Condition1_Other,Condition1_PosN,Condition1_RRAn,Condition2_Norm,Condition2_Other,Condition2_PosN,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_BrkFace,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Other,Exterior1st_Plywood,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_BrkFace,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_None,PavedDrive_P,PavedDrive_Y,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_None,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,65.00,8450,7,5,196.00,706.00,0.00,150.00,856.00,856,854,0,1710,1.00,0.00,2,1,3,1,8,0,2.00,548.00,0,61,0,0,0,0,0,2,208500.00,4,3,4,3,5,4,0,3,3,0,2,3,6,1,0,17,17,17.00,12,12.25,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0
1,80.00,9600,6,8,0.00,978.00,0.00,284.00,1262.00,1262,0,0,1262,0.00,1.00,2,0,3,1,6,1,2.00,460.00,298,0,0,0,0,0,0,5,181500.00,3,3,4,3,5,3,3,3,3,0,2,3,5,1,3,44,44,44.00,13,12.11,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,

In [28]:
print(df1.dtypes.astype(str).value_counts())

uint8      152
int64       39
float64     13
dtype: int64


## Export

In [44]:
# Write to CSV
df1.to_csv('data/processed_data.csv', index = False)